In [ ]:
# Name: Integration with Legacy & Niche Systems
# Author: Rod Villalobos
# Purpose: Show an AI prototype 

import numpy as np, pandas as pd, matplotlib.pyplot as plt
from datetime import datetime

np.random.seed(23)

# --- 1) Simulate a LEGACY dataset (weird names/formats/units) ---
N = 120
dates_raw = []
for _ in range(N):
    if np.random.rand() < 0.7:
        # mm/dd/yyyy
        d = pd.Timestamp("2024-01-01") + pd.to_timedelta(np.random.randint(0, 365), unit="D")
        dates_raw.append(d.strftime("%m/%d/%Y"))
    else:
        # yyyy-mm-dd
        d = pd.Timestamp("2024-01-01") + pd.to_timedelta(np.random.randint(0, 365), unit="D")
        dates_raw.append(d.strftime("%Y-%m-%d"))

legacy_df = pd.DataFrame({
    "PID": np.arange(1, N + 1),
    "AGE_YRS": np.random.choice(list(range(18, 90)) + [999, -4], size=N),
    "SEX_CD": np.random.choice(["M", "F", "U", "?"], size=N, p=[0.46, 0.46, 0.04, 0.04]),
    "ENROLL_DT": dates_raw,
    "HT_IN": np.random.normal(67, 3.5, size=N),      # inches
    "WT_LB": np.random.normal(165, 30, size=N),      # pounds
    "SITE": np.random.choice(["LA1", "NY2", "TX3", "??"], size=N, p=[0.4, 0.35, 0.2, 0.05]),
})
# Inject legacy quirks
legacy_df.loc[np.random.choice(N, 4, replace=False), "HT_IN"] = np.nan
legacy_df.loc[np.random.choice(N, 3, replace=False), "WT_LB"] = np.nan
legacy_df["UNEXPECTED_LEGACY_FLAG"] = np.random.choice([0, 1], size=N, p=[0.9, 0.1])

# --- 2) Define target MODERN schema + mappings ---
target_columns = ["patient_id", "age", "sex", "date_enrolled", "height_cm", "weight_kg", "site_code"]

column_map = {
    "PID": "patient_id",
    "AGE_YRS": "age",
    "SEX_CD": "sex",
    "ENROLL_DT": "date_enrolled",
    "HT_IN": "height_cm",     # will convert inches -> cm
    "WT_LB": "weight_kg",     # will convert pounds -> kg
    "SITE": "site_code",
}

# Site normalization (legacy codes to canonical)
site_map = {"LA1": "LAX", "NY2": "JFK", "TX3": "DFW"}
# Sex normalization (legacy)
sex_norm = {"M": "M", "F": "F", "U": None, "?": None}

# --- 3) Transform legacy -> modern ---
# 3a) Basic column renaming (only those in the map)
mapped = legacy_df[[c for c in legacy_df.columns if c in column_map]].rename(columns=column_map)

# 3b) Convert units + normalize enums + parse dates
def inches_to_cm(x):
    return x * 2.54 if pd.notna(x) else np.nan

def pounds_to_kg(x):
    return x * 0.45359237 if pd.notna(x) else np.nan

def parse_mixed_date(s):
    # try mm/dd/yyyy then yyyy-mm-dd
    try:
        return pd.to_datetime(s, format="%m/%d/%Y", errors="raise")
    except Exception:
        try:
            return pd.to_datetime(s, format="%Y-%m-%d", errors="raise")
        except Exception:
            return pd.NaT

# Keep counters for transform diagnostics
date_failures = 0
for i, s in mapped["date_enrolled"].items():
    dt = parse_mixed_date(s)
    if pd.isna(dt):
        date_failures += 1
    mapped.at[i, "date_enrolled"] = dt

mapped["height_cm"] = mapped["height_cm"].apply(inches_to_cm)
mapped["weight_kg"] = mapped["weight_kg"].apply(pounds_to_kg)
mapped["sex"] = mapped["sex"].map(sex_norm).astype("object")
mapped["site_code"] = mapped["site_code"].map(site_map).astype("object")

# 3c) Ensure target schema presence + order
for col in target_columns:
    if col not in mapped.columns:
        mapped[col] = np.nan
mapped = mapped[target_columns]

# --- 4) Mapping coverage + transform diagnostics ---
legacy_cols = set(legacy_df.columns)
mapped_from_legacy = set(column_map.keys())
unmapped_legacy_cols = sorted(list(legacy_cols - mapped_from_legacy))
mapping_coverage = round(100.0 * len(mapped_from_legacy) / len(legacy_cols), 1)

unit_null_h_cm = int(mapped["height_cm"].isna().sum())
unit_null_w_kg = int(mapped["weight_kg"].isna().sum())

# --- 5) Validation on MODERN dataset ---
ENUMS = {"sex": {"M", "F"}, "site_code": {"LAX", "JFK", "DFW"}}
RANGES = {"age": (18, 90), "height_cm": (120.0, 220.0), "weight_kg": (35.0, 200.0)}

def validate(df: pd.DataFrame) -> dict:
    report = {}
    report["missing_columns"] = [c for c in target_columns if c not in df.columns]
    report["unexpected_columns"] = [c for c in df.columns if c not in target_columns]
    report["nulls_per_column"] = {c: int(df[c].isna().sum()) for c in target_columns}
    report["duplicate_rows_on_patient_id"] = int(df.duplicated(subset=["patient_id"], keep=False).sum())

    enum_viol = {}
    for col, allowed in ENUMS.items():
        if col in df.columns:
            enum_viol[col] = int((df[col].notna() & (~df[col].isin(allowed))).sum())
    report["enum_violations"] = enum_viol

    range_viol = {}
    for col, (low, high) in RANGES.items():
        if col in df.columns:
            mask = df[col].notna() & ((df[col] < low) | (df[col] > high))
            range_viol[col] = int(mask.sum())
    report["range_violations"] = range_viol
    return report

validation = validate(mapped)

summary = {
    # Mapping / transform
    "mapping_coverage_percent": mapping_coverage,
    "unmapped_legacy_cols": len(unmapped_legacy_cols),
    "date_parse_failures": int(date_failures),
    "height_cm_nulls_after_conv": unit_null_h_cm,
    "weight_kg_nulls_after_conv": unit_null_w_kg,
    # Validation on modern
    "unexpected_columns_in_modern": len(validation["unexpected_columns"]),
    "total_nulls_modern": sum(validation["nulls_per_column"].values()),
    "dup_patient_id_modern": validation["duplicate_rows_on_patient_id"],
    "enum_violations_total": sum(validation["enum_violations"].values()),
    "range_violations_total": sum(validation["range_violations"].values()),
}

print("=== MAPPING / TRANSFORM SUMMARY ===")
print({
    "mapping_coverage_percent": summary["mapping_coverage_percent"],
    "unmapped_legacy_cols_list": unmapped_legacy_cols,
    "date_parse_failures": summary["date_parse_failures"],
    "height_cm_nulls_after_conv": summary["height_cm_nulls_after_conv"],
    "weight_kg_nulls_after_conv": summary["weight_kg_nulls_after_conv"],
}, "\n")

print("=== VALIDATION REPORT (MODERN SCHEMA) ===")
print(validation, "\n")

print("=== SUMMARY (KEY COUNTS) ===")
print(summary, "\n")

# --- 6) Quick visualization of key counts ---
plot_keys = [
    "unmapped_legacy_cols",
    "date_parse_failures",
    "height_cm_nulls_after_conv",
    "weight_kg_nulls_after_conv",
    "enum_violations_total",
    "range_violations_total",
]
plt.figure(figsize=(8,4))
plt.bar(plot_keys, [summary[k] for k in plot_keys])
plt.title("Integration Issues — Mapping & Validation Summary")
plt.ylabel("Count")
plt.xticks(rotation=20)
plt.tight_layout()
plt.show()

